## Cell-Nucleus Correlation

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

In [ ]:
# Conversions
# 1 frame is 15 minutes
# 10.5 pixels is 1 um 

In [ ]:
def preprocess(channel):
    
    # Blur, threashold, and close as preprocessing 
    blurred = cv2.GaussianBlur(channel, (9, 9), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_OTSU)
    kernel = np.ones((3,3), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=4)
    
    return closing

def findBoundaries(channel):
    preprocessed = preprocess(channel)
    contours, _ = cv2.findContours(preprocessed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter out contours that are too small or match the image boundary
    good_contours = []
    img_height, img_width = channel.shape
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Exclude contours that are at the boundary of the image or too small
        if (x > 1 and y > 1 and x + w < img_width - 1 and y + h < img_height - 1) and cv2.contourArea(contour) > 100:  # Threshold
            good_contours.append(contour)
    
    # Assuming the largest remaining contour is the cell/nucleus
    if good_contours:
        largest_contour = max(good_contours, key=cv2.contourArea)
        return largest_contour
    else:
        return None 

def findCentroid(contour):
    M = cv2.moments(contour)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    return cx, cy

def compute_velocity_angles(df):
    # Calculate differences between consecutive frames for x and y coordinates
    df['cell_dx'] = df['cell_centroid_x'].diff()
    df['cell_dy'] = df['cell_centroid_y'].diff()
    df['nucleus_dx'] = df['nucleus_centroid_x'].diff()
    df['nucleus_dy'] = df['nucleus_centroid_y'].diff()
    
    # Calculate angles in radians
    df['cell_angle_rad'] = np.arctan2(df['cell_dy'], df['cell_dx'])
    df['nucleus_angle_rad'] = np.arctan2(df['nucleus_dy'], df['nucleus_dx'])
    
    # Convert angles from radians to degrees for easier interpretation
    df['cell_angle_deg'] = np.degrees(df['cell_angle_rad'])
    df['nucleus_angle_deg'] = np.degrees(df['nucleus_angle_rad'])

    # Calculate velocity
    df['cell_vel'] = np.sqrt(np.square(df['cell_dy']) + np.square(df['cell_dx']))
    df['nucleus_vel'] = np.sqrt(np.square(df['nucleus_dy']) + np.square(df['nucleus_dx']))
    
    # Convert frames to min
    df['time'] = df['frame'] * 15
    
    # Convert velocity to um/min 
    df['cell_vel_scaled'] = df['cell_vel'] / 15 / 10.5
    df['nucleus_vel_scaled'] = df['nucleus_vel'] / 15 / 10.5
    
    return df

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    centroids = {'frame': [], 'cell_centroid_x': [], 'cell_centroid_y': [], 'nucleus_centroid_x': [], 'nucleus_centroid_y': []}
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        
        # Split channels
        red_channel = frame[:, :, 2]
        green_channel = frame[:, :, 1]
            
        # Find boundaries
        cell_contour = findBoundaries(green_channel)
        nucleus_contour = findBoundaries(red_channel)
        
        # Find centroids
        cell_centroid = findCentroid(cell_contour)
        nucleus_centroid = findCentroid(nucleus_contour)
        
        centroids['frame'].append(frame_number)
        centroids['cell_centroid_x'].append(cell_centroid[0])
        centroids['cell_centroid_y'].append(cell_centroid[1])
        centroids['nucleus_centroid_x'].append(nucleus_centroid[0])
        centroids['nucleus_centroid_y'].append(nucleus_centroid[1])
            
    cap.release()
    df = pd.DataFrame(centroids)
    
    return compute_velocity_angles(df)


In [ ]:
def plotting_data(df):
    plt.style.use('classic') 
    
    # Nucleus and cell speed
    
    plt.figure(figsize=(6, 4))

    plt.plot(df['frame'] * 15, df['cell_vel'] / 15 / 10.5, label='Whole Cell', marker='s', linestyle='--', markersize=5, linewidth=1)
    plt.plot(df['frame'] * 15, df['nucleus_vel'] / 15 / 10.5, label='Nucleus', marker='o', linestyle='-', markersize=5, linewidth=1)
    
    plt.title('Cell and Nucleus Velocity Over Time') 
    plt.xlabel('Time (min)') 
    plt.ylabel('Vel (μm/min)')  
    plt.legend(loc='upper left')  

    plt.tight_layout()  
    plt.show()
    
    
    # Nucleus and cell angles

    plt.figure(figsize=(6, 4))

    plt.plot(df['frame'] * 15, df['cell_angle_deg'], label='Whole Cell', marker='s', linestyle='--', markersize=5, linewidth=1, color='green')
    plt.plot(df['frame'] * 15, df['nucleus_angle_deg'], label='Nucleus', marker='o', linestyle='-', markersize=5, linewidth=1)
    
    plt.title('Cell and Nucleus Velocity Direction Over Time')
    plt.xlabel('Time (min)') 
    plt.ylabel('Velocity Direction (deg)')
    plt.legend(loc='upper left')

    plt.tight_layout()  
    plt.show()

In [ ]:
def correlation(df):

    nucleus_angles = df['nucleus_angle_deg']
    cell_angles = df['cell_angle_deg']
    
    nucleus_vel = df['nucleus_vel']
    cell_vel = df['cell_vel']

    # Compute Spearman's rank correlation
    angle, p_value = spearmanr(nucleus_angles, cell_angles, nan_policy='omit')
    vel, p_value = spearmanr(nucleus_vel, cell_vel, nan_policy='omit')

    return vel, angle

# Results

In [ ]:
video_path = 'Actin-nucleus co-stain 3.mp4'
df = process_video(video_path)
plotting_data(df)
print(correlation(df))

In [ ]:
df.to_csv("CellNucleusCorrelation.csv")

In [ ]:
df